In [53]:
import torch

if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        raise Exception("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        raise Exception("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

In [54]:
import numpy as np
from dlordinal.datasets import FGNet
from dlordinal.losses import TriangularCrossEntropyLoss
from dlordinal.metrics import amae, mmae
from skorch import NeuralNetClassifier
from torch import nn
from torch.optim import Adam
from torchvision import models
from torchvision.transforms import Compose, ToTensor

# Download the FGNet dataset
fgnet_train = FGNet(
    root="./datasets",
    train=True,
    target_transform=np.array,
    transform=Compose([ToTensor()]),
)
fgnet_test = FGNet(
    root="./datasets",
    train=False,
    target_transform=np.array,
    transform=Compose([ToTensor()]),
)

num_classes_fgnet = len(fgnet_train.classes)

# Model
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes_fgnet)

# Loss function
loss_fn = TriangularCrossEntropyLoss(num_classes=num_classes_fgnet)

# Skorch estimator
estimator = NeuralNetClassifier(
    module=model,
    criterion=loss_fn,
    optimizer=Adam,
    lr=1e-3,
    max_epochs=25,
    device='mps',
)

estimator.fit(X=fgnet_train, y=fgnet_train.targets)
train_probs = estimator.predict_proba(fgnet_train)
test_probs = estimator.predict_proba(fgnet_test)

# Metrics
amae_metric = amae(np.array(fgnet_test.targets), test_probs)
mmae_metric = mmae(np.array(fgnet_test.targets), test_probs)
print(f"Test AMAE: {amae_metric}, Test MMAE: {mmae_metric}")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.6924       0.3292        2.0299  8.2437
      2        0.9279       0.3292        2.7551  2.3382
      3        0.6834       0.4596        2.2886  2.3409
      4        0.5570       0.3851        2.2740  2.3239
      5        0.4407       0.5342        1.6565  2.3237
      6        0.4043       0.5528        1.3586  2.3223
      7        0.3782       0.5528        1.3014  2.3161
      8        0.3647       0.5839        1.2299  2.3190
      9        0.3567       0.5839        1.1925  2.3138
     10        0.3520       0.5901        1.1746  2.3358
     11        0.3497       0.6025        1.1682  2.3695
     12        0.3479       0.6025        1